In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

df = pd.read_csv('dataset.csv')
df.head()
pd.set_option('future.no_silent_downcasting', True)
df.isnull().sum()
DT_bmi_pipe = Pipeline( steps=[
                               ('scale',StandardScaler()),
                               ('lr',DecisionTreeRegressor(random_state=42))
                              ])
X = df[['age','gender','bmi']].copy()
X.gender = X.gender.replace({'Male':0,'Female':1,'Other':-1}).astype(np.int64)

Missing = X[X.bmi.isna()]
X = X[~X.bmi.isna()]
Y = X.pop('bmi')
DT_bmi_pipe.fit(X,Y)
predicted_bmi = pd.Series(DT_bmi_pipe.predict(Missing[['age','gender']]),index=Missing.index)
df.loc[Missing.index,'bmi'] = predicted_bmi

In [20]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
df['gender'] = df['gender'].replace({'Male':0,'Female':1,'Other':-1}).astype(np.int64)
df['Residence_type'] = df['Residence_type'].replace({'Rural':0,'Urban':1}).astype(np.int64)
df['work_type'] = df['work_type'].replace({'Private':0,'Self-employed':1,'Govt_job':2,'children':-1,'Never_worked':-2}).astype(np.int64)
x = df[['gender','age','hypertension','heart_disease','work_type','avg_glucose_level','bmi']]
y = df['stroke']
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.3, random_state=42)
oversample = SMOTE()
x_train_resh, y_train_resh = oversample.fit_resample(x_train, y_train.to_numpy())
logreg_pipeline = Pipeline(steps = [('scale',StandardScaler()),('LR',LogisticRegression(random_state=42))])
logreg_cv = cross_val_score(logreg_pipeline,x_train_resh,y_train_resh,cv=10,scoring='f1')
print('Logistic Regression mean :',cross_val_score(logreg_pipeline,x_train_resh,y_train_resh,cv=10,scoring='f1').mean())
logreg_pipeline.fit(x_train_resh,y_train_resh)
logreg_pred = logreg_pipeline.predict(x_test)
logreg_cm = confusion_matrix(y_test, logreg_pred)
logreg_f1 = f1_score(y_test,logreg_pred)
print('LR mean :',logreg_f1)
logreg_pipeline = Pipeline(steps = [('scale',StandardScaler()),('LR',LogisticRegression(C=0.1,penalty='l2',random_state=42))])

logreg_pipeline.fit(x_train_resh,y_train_resh)
logreg_tuned_pred = logreg_pipeline.predict(x_test)
print(f1_score(y_test,logreg_tuned_pred), accuracy_score(y_test,logreg_tuned_pred))

Logistic Regression mean : 0.8221748914949153
LR mean : 0.19423076923076923
0.1946564885496183 0.7640480849874196
